## Load alphase from directory into dictionary sturct to pickle

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pickle

In [4]:
def read_alpha(filename):
    with open(filename,'r') as myfile:
        contents= myfile.read()
    alpha_list = contents.split()
    alpha_list = np.array(alpha_list).astype(float)
    n , nx, ny, nz = alpha_list[0:4].astype(int)
    alpha_list= np.delete(alpha_list, [0,1,2,3])
    a=np.zeros([nx,ny,nz])
    point = -1
    for iz in range(0,nz):
        for iy in range(0,ny):
            for ix in range(0,nx):
                point += 1
                a[ix,iy,iz]=alpha_list[point]
    
    if point+1==n:
        return alpha_list , a
    else:
        print ("Error: lengths dont match")
        print (point, n)
        return 

In [5]:
def load_from_dir(base_path,exclude_list=[".","_",","],pass_suffix='.out',pass_format_flag=False,pass_delim='_',pass_split_num=2):
    #create dictionary
    pbc={}
    #itterate over folders in base_path
    for folder in base_path.iterdir():
        #ignore files
        if folder.is_dir():
            #ignore hidden and special folders
            if folder.name[0] not in exclude_list:
                temp_dir={}
                temp_dir=load_out_files(folder,suffix=pass_suffix,format_flag=pass_format_flag,delim=pass_delim,split_num=pass_split_num)
                parts = folder.stem.split('(') #pass_delim
                #del parts[0:3]
                key=parts[0]
                #key= folder.stem
#                 if len(parts)>1:
#                     for item in parts[1:]:
#                         key += "_"
#                         key += item
                pbc[key]=temp_dir
    return pbc 


In [6]:

def load_out_files(path,suffix='.out',format_flag=False,delim='_',split_num=2):
    #takes path and return dictionary with keys as filenames and 
    path_object=pathlib.Path(path)
    obj_dict={}
    for file in path_object.iterdir():
        if file.name[0] not in [".",",","_"]:
            if file.suffix==suffix:
                key=file.stem
                if format_flag==True:
                    parts=key.split(delim,split_num)
                    key=parts[-1]
                #parts=file.stem.split("_")
                #key = parts[-1]
                
                obj_dict[key]=read_alpha(file)
        
    return obj_dict

In [7]:
def write_alpha_file(alpha_list,filename,seperator='    '):
    nx = round(len(alpha_list)**(1.0/3.0))
    ny = nx 
    nz = nx

    with open(filename,'w') as myfile:
        myfile.writelines('a'+seperator+'b'+seperator+'c'+seperator+'charge\n')
        point = -1
        for iz in range(1,nz+1):
            for iy in range(1,ny+1):
                for ix in range(1,nx+1):
                    point += 1
                    line = str(ix)+seperator+str(iy)+seperator+str(iz)+\
                        seperator+str(alpha_list[point])+' '+ '\n'
                    myfile.writelines(line)
                

In [8]:
def folder_alpha_write(data,seed,path,seperator='    '):
    for alpha in data[seed].keys():
        path_name = pathlib.Path.joinpath(path,str(alpha)+'.charg_frm')
        write_alpha_file(data[seed][alpha][0],path_name,seperator)

In [9]:
base_path = pathlib.Path.cwd()

In [10]:
y = load_from_dir(base_path)

In [11]:
y.keys()

dict_keys(['Si', 'LiF', 'Na', 'Li', 'C', 'Ag', 'MgO'])

In [17]:
y['Si']['alpha_1'][1]

array([[[1.07071285, 1.10894438, 1.23014065, ..., 1.45279353,
         1.23014062, 1.1089444 ],
        [1.10894438, 1.18898572, 1.36166407, ..., 1.36168713,
         1.18897349, 1.10894973],
        [1.23014065, 1.36166407, 1.60074036, ..., 1.36168921,
         1.23013283, 1.18838822],
        ...,
        [1.45279353, 1.36168713, 1.36168921, ..., 2.41728738,
         1.95930373, 1.6460395 ],
        [1.23014062, 1.18897349, 1.23013283, ..., 1.95930373,
         1.59520886, 1.35978914],
        [1.1089444 , 1.10894973, 1.18838822, ..., 1.6460395 ,
         1.35978914, 1.1884004 ]],

       [[1.10894438, 1.18898572, 1.36166407, ..., 1.36168713,
         1.18897349, 1.10894973],
        [1.18898572, 1.31740228, 1.55164187, ..., 1.31740236,
         1.18896936, 1.14827619],
        [1.36166407, 1.55164187, 1.8633668 , ..., 1.36170197,
         1.2726928 , 1.27268951],
        ...,
        [1.36168713, 1.31740236, 1.36170197, ..., 2.12934699,
         1.74938872, 1.50040887],
        [1.1

In [35]:
# pickling data into object file
with open('data.obj','wb') as myfile:
     pickle.dump(y,myfile)

In [12]:
# example of how to load data 
with open('data.obj','rb') as myfile:
     z= pickle.load(myfile)

In [58]:
# writing alpha values

In [13]:
temp_path = base_path.joinpath('Li(bcc-W)')
folder_alpha_write(y,'Li',temp_path)

In [14]:
temp_path = base_path.joinpath('Na(bcc-W)')
folder_alpha_write(y,'Na',temp_path)

In [15]:
temp_path = base_path.joinpath('LiF(NaCl')
folder_alpha_write(y,'LiF',temp_path)

In [16]:
temp_path = base_path.joinpath('MgO(NaCl)')
folder_alpha_write(y,'MgO',temp_path)

In [26]:
temp_path = base_path.joinpath('Si(diamond)')
folder_alpha_write(y,'Si',temp_path)

In [27]:
temp_path = base_path.joinpath('C(diamond)')
folder_alpha_write(y,'C',temp_path)

In [28]:
temp_path = base_path.joinpath('Ag(fcc)')
folder_alpha_write(y,'Ag',temp_path)